# E2LSH Pipeline評価 - DuckDB + E5モデル

## 目的

E2LSHをパイプラインに統合し、DuckDBのHNSW検索と比較評価する。

## 構成

1. DuckDBからE5モデルのベクトルを読み込み
2. E2LSHインデックス構築
3. HNSW vs E2LSH Cascade の比較
4. Recall@10、レイテンシ評価

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import time

from src.db import VectorDatabase
from src.pipeline import HNSWSearcher, E2LSHCascadeSearcher

## 1. DuckDBからデータ読み込み

In [2]:
# DuckDBからE5モデルのデータを読み込み
db_path = Path('../data/sample_vectors.duckdb')
db = VectorDatabase(db_path=db_path)
db.initialize()

print(f'Total documents: {db.count():,}')

# 全データ取得
all_docs = db.get_all()
print(f'Columns: {all_docs.columns.tolist()}')

Total documents: 10,000
Columns: ['id', 'text', 'vector', 'simhash', 'lsh_chunks']


In [3]:
# ベクトルとメタデータを抽出
vectors = np.stack(all_docs['vector'].values).astype(np.float32)
texts = all_docs['text'].tolist()
ids = all_docs['id'].tolist()

print(f'Vectors shape: {vectors.shape}')
print(f'Norm range: [{np.linalg.norm(vectors, axis=1).min():.4f}, {np.linalg.norm(vectors, axis=1).max():.4f}]')

Vectors shape: (10000, 1024)
Norm range: [1.0000, 1.0000]


## 2. 検索器の初期化

In [4]:
# HNSW検索器（DuckDB）
hnsw_searcher = HNSWSearcher(db)

# E2LSH検索器（パラメータ: 前回の実験で最適だった値）
e2lsh_searcher = E2LSHCascadeSearcher(
    vectors=vectors,
    texts=texts,
    ids=ids,
    w=8.0,      # バケット幅
    k=4,        # ハッシュ関数数/テーブル
    num_tables=8,  # テーブル数
    seed=42,
)

print('検索器を初期化しました')
print(f'  E2LSH: w=8.0, k=4, L=8')

検索器を初期化しました
  E2LSH: w=8.0, k=4, L=8


## 3. 単一クエリでの動作確認

In [5]:
# 最初のドキュメントをクエリとして使用
query_vec = vectors[0]
query_id = ids[0]
query_text = texts[0]

print(f'Query ID: {query_id}')
print(f'Query Text: {query_text[:50]}...')

Query ID: 0
Query Text: 石上私淑言...


In [6]:
# HNSW検索
hnsw_results, hnsw_time = hnsw_searcher.search(query_vec, top_k=10)

print('=== HNSW Results ===')
print(f'Time: {hnsw_time:.2f} ms')
for r in hnsw_results[:5]:
    print(f'  id={r.id}, score={r.score:.4f}, text={r.text[:30]}...')

=== HNSW Results ===
Time: 67.87 ms
  id=0, score=1.0000, text=石上私淑言...
  id=1186, score=0.8509, text=Template:皇室典範/blockquote@style...
  id=3632, score=0.8497, text=石川滋...
  id=240, score=0.8470, text=愚直師侃...
  id=754, score=0.8440, text=秘曲 笑傲江湖...


In [7]:
# E2LSH検索
e2lsh_results, e2lsh_metrics = e2lsh_searcher.search(query_vec, top_k=10)

print('=== E2LSH Cascade Results ===')
print(f'Total time: {e2lsh_metrics.total_time_ms:.2f} ms')
print(f'Step1 candidates: {e2lsh_metrics.step1_candidates}')
print()
for r in e2lsh_results[:5]:
    print(f'  id={r.id}, score={r.score:.4f}, text={r.text[:30]}...')

=== E2LSH Cascade Results ===
Total time: 26.63 ms
Step1 candidates: 100

  id=0, score=1.0000, text=石上私淑言...
  id=1186, score=0.8509, text=Template:皇室典範/blockquote@style...
  id=3632, score=0.8497, text=石川滋...
  id=240, score=0.8470, text=愚直師侃...
  id=754, score=0.8440, text=秘曲 笑傲江湖...


In [8]:
# Recall計算
hnsw_ids = set(r.id for r in hnsw_results)
e2lsh_ids = set(r.id for r in e2lsh_results)
recall = len(hnsw_ids & e2lsh_ids) / len(hnsw_ids)

print(f'Recall@10: {recall:.2f}')
print(f'HNSW IDs: {sorted(hnsw_ids)}')
print(f'E2LSH IDs: {sorted(e2lsh_ids)}')
print(f'Overlap: {sorted(hnsw_ids & e2lsh_ids)}')

Recall@10: 1.00
HNSW IDs: [0, 240, 486, 754, 1186, 1368, 2880, 3632, 4164, 4454]
E2LSH IDs: [0, 240, 486, 754, 1186, 1368, 2880, 3632, 4164, 4454]
Overlap: [0, 240, 486, 754, 1186, 1368, 2880, 3632, 4164, 4454]


## 4. 複数クエリでの評価

In [9]:
import random

# 100件のクエリをランダムサンプリング
random.seed(42)
num_queries = 100
query_indices = random.sample(range(len(vectors)), num_queries)

print(f'評価クエリ数: {num_queries}')

評価クエリ数: 100


In [10]:
def evaluate_searchers(query_indices, hnsw_searcher, e2lsh_searcher, vectors):
    """両検索器を評価"""
    results = {
        'hnsw_times': [],
        'e2lsh_times': [],
        'recalls': [],
        'e2lsh_candidates': [],
    }
    
    for idx in query_indices:
        query_vec = vectors[idx]
        
        # HNSW検索
        hnsw_results, hnsw_time = hnsw_searcher.search(query_vec, top_k=10)
        hnsw_ids = set(r.id for r in hnsw_results)
        
        # E2LSH検索
        e2lsh_results, e2lsh_metrics = e2lsh_searcher.search(query_vec, top_k=10)
        e2lsh_ids = set(r.id for r in e2lsh_results)
        
        # Recall
        recall = len(hnsw_ids & e2lsh_ids) / len(hnsw_ids) if hnsw_ids else 0
        
        results['hnsw_times'].append(hnsw_time)
        results['e2lsh_times'].append(e2lsh_metrics.total_time_ms)
        results['recalls'].append(recall)
        results['e2lsh_candidates'].append(e2lsh_metrics.step1_candidates)
    
    return results

# 評価実行
eval_results = evaluate_searchers(query_indices, hnsw_searcher, e2lsh_searcher, vectors)

In [11]:
# 結果サマリー
print('=' * 60)
print('          HNSW vs E2LSH 評価結果 (E5モデル, 10,000件)')
print('=' * 60)
print(f"\n{'指標':<25} {'HNSW':>15} {'E2LSH':>15}")
print('-' * 57)
print(f"{'平均レイテンシ (ms)':<25} {np.mean(eval_results['hnsw_times']):>15.2f} {np.mean(eval_results['e2lsh_times']):>15.2f}")
print(f"{'P50 レイテンシ (ms)':<25} {np.percentile(eval_results['hnsw_times'], 50):>15.2f} {np.percentile(eval_results['e2lsh_times'], 50):>15.2f}")
print(f"{'P99 レイテンシ (ms)':<25} {np.percentile(eval_results['hnsw_times'], 99):>15.2f} {np.percentile(eval_results['e2lsh_times'], 99):>15.2f}")
print(f"{'Recall@10':<25} {'1.000':>15} {np.mean(eval_results['recalls']):>15.3f}")
print(f"{'平均候補数':<25} {len(vectors):>15} {np.mean(eval_results['e2lsh_candidates']):>15.1f}")
print('=' * 60)

          HNSW vs E2LSH 評価結果 (E5モデル, 10,000件)

指標                                   HNSW           E2LSH
---------------------------------------------------------
平均レイテンシ (ms)                        11.52           26.59
P50 レイテンシ (ms)                      11.03           25.20
P99 レイテンシ (ms)                      18.13           41.19
Recall@10                           1.000           0.996
平均候補数                               10000           100.0


## 5. E2LSHパラメータチューニング

In [12]:
# 異なるパラメータでの評価
param_configs = [
    {'w': 4.0, 'k': 4, 'num_tables': 4},
    {'w': 4.0, 'k': 4, 'num_tables': 8},
    {'w': 8.0, 'k': 4, 'num_tables': 8},
    {'w': 8.0, 'k': 4, 'num_tables': 16},
    {'w': 16.0, 'k': 4, 'num_tables': 8},
]

param_results = []

for config in param_configs:
    # 検索器を作成
    searcher = E2LSHCascadeSearcher(
        vectors=vectors,
        texts=texts,
        ids=ids,
        **config,
        seed=42,
    )
    
    # 評価
    recalls = []
    times = []
    candidates = []
    
    for idx in query_indices:
        query_vec = vectors[idx]
        
        # HNSW (Ground Truth)
        hnsw_results, _ = hnsw_searcher.search(query_vec, top_k=10)
        hnsw_ids = set(r.id for r in hnsw_results)
        
        # E2LSH
        e2lsh_results, metrics = searcher.search(query_vec, top_k=10)
        e2lsh_ids = set(r.id for r in e2lsh_results)
        
        recalls.append(len(hnsw_ids & e2lsh_ids) / len(hnsw_ids) if hnsw_ids else 0)
        times.append(metrics.total_time_ms)
        candidates.append(metrics.step1_candidates)
    
    param_results.append({
        'w': config['w'],
        'k': config['k'],
        'L': config['num_tables'],
        'recall': np.mean(recalls),
        'latency_ms': np.mean(times),
        'candidates': np.mean(candidates),
    })
    
    print(f"w={config['w']}, k={config['k']}, L={config['num_tables']}: "
          f"Recall={np.mean(recalls):.3f}, Latency={np.mean(times):.2f}ms")

w=4.0, k=4, L=4: Recall=0.996, Latency=26.74ms
w=4.0, k=4, L=8: Recall=0.996, Latency=26.67ms
w=8.0, k=4, L=8: Recall=0.996, Latency=26.73ms
w=8.0, k=4, L=16: Recall=0.996, Latency=27.27ms
w=16.0, k=4, L=8: Recall=0.996, Latency=26.63ms


In [13]:
# パラメータ比較表
df_params = pd.DataFrame(param_results)
print('\n=== E2LSH パラメータ比較 ===')
print(df_params.to_string(index=False))


=== E2LSH パラメータ比較 ===
   w  k  L  recall  latency_ms  candidates
 4.0  4  4   0.996   26.744060       100.0
 4.0  4  8   0.996   26.666629       100.0
 8.0  4  8   0.996   26.728597       100.0
 8.0  4 16   0.996   27.274691       100.0
16.0  4  8   0.996   26.627271       100.0


## 6. インデックス構築時間の計測

In [14]:
# インデックス構築時間
from src.e2lsh import E2LSHHasher, E2LSHIndex

configs = [
    {'w': 8.0, 'k': 4, 'num_tables': 8},
    {'w': 8.0, 'k': 4, 'num_tables': 16},
]

print('=== インデックス構築時間 ===')
for config in configs:
    hasher = E2LSHHasher(
        dim=vectors.shape[1],
        w=config['w'],
        k=config['k'],
        num_tables=config['num_tables'],
        seed=42,
    )
    index = E2LSHIndex(hasher)
    
    start = time.perf_counter()
    index.build(vectors)
    build_time = (time.perf_counter() - start) * 1000
    
    print(f"w={config['w']}, k={config['k']}, L={config['num_tables']}: {build_time:.1f} ms")

=== インデックス構築時間 ===
w=8.0, k=4, L=8: 174.8 ms
w=8.0, k=4, L=16: 370.9 ms


## 7. 評価レポート

In [15]:
# 最終比較
best_config = max(param_results, key=lambda x: x['recall'])

print('=' * 70)
print('               E2LSH Pipeline 評価レポート')
print('=' * 70)
print(f'''
【データセット】
  モデル: E5-large (multilingual-e5-large)
  件数: {len(vectors):,}
  次元: {vectors.shape[1]}

【最適パラメータ】
  w = {best_config['w']}
  k = {best_config['k']}
  L = {best_config['L']}

【性能比較】
  | 指標 | HNSW | E2LSH |
  |------|------|-------|
  | Recall@10 | 1.000 | {best_config['recall']:.3f} |
  | 平均レイテンシ | {np.mean(eval_results['hnsw_times']):.2f} ms | {best_config['latency_ms']:.2f} ms |
  | 候補数 | {len(vectors):,} | {best_config['candidates']:.0f} |

【考察】
  - E2LSHはメモリ内インデックスで動作
  - 10,000件規模ではHNSWの方がシンプルで高速
  - 100万件以上でE2LSHの優位性が出る可能性
''')
print('=' * 70)

               E2LSH Pipeline 評価レポート

【データセット】
  モデル: E5-large (multilingual-e5-large)
  件数: 10,000
  次元: 1024

【最適パラメータ】
  w = 4.0
  k = 4
  L = 4

【性能比較】
  | 指標 | HNSW | E2LSH |
  |------|------|-------|
  | Recall@10 | 1.000 | 0.996 |
  | 平均レイテンシ | 11.52 ms | 26.74 ms |
  | 候補数 | 10,000 | 100 |

【考察】
  - E2LSHはメモリ内インデックスで動作
  - 10,000件規模ではHNSWの方がシンプルで高速
  - 100万件以上でE2LSHの優位性が出る可能性



In [16]:
# データベース接続を閉じる
db.close()
print('Done!')

Done!
